In [1]:
from models.detector_model.model import ObjectDetectionModel
from models.detector_model.processor import TrainingProcessor
from models.detector_model.data_utils import TrainingDataset, COCOProcessor
from models.detector_model.loss1 import ObjectDetectionLoss, train_with_monitoring, create_balanced_loss
from torch.utils.data import DataLoader
import torch
import torch.optim as optim

grouped_classes = {
        "Metal": [
            "Metal bottle cap", "Metal lid", "Drink can", "Pop tab", "Scrap metal",
            "Food Can", "Aluminium blister pack", "Aluminium foil", "Aerosol"
        ],
        "Plastic": [
            "Plastic bottle cap", "Other plastic wrapper", "Six pack rings",
            "Single-use carrier bag", "Plastic straw", "Plastic glooves",
            "Plastic utensils", "Disposable plastic cup", "Other plastic bottle",
            "Tupperware", "Spread tub", "Garbage bag", "Other plastic container",
            "Other plastic", "Rope & strings", "Other plastic cup", "Plastic film",
            "Polypropylene bag", "Plastic lid", "Clear plastic bottle", "Squeezable tube",
            "Carded blister pack", "Crisp packet", "Meal carton"
        ],
        "Paper": [
            "Paper cup", "Paper bag", "Normal paper", "Paper straw", "Tissues",
            "Toilet tube", "Wrapping paper", "Pizza box", "Magazine paper",
            "Corrugated carton", "Egg carton", "Other carton", "Drink carton"
        ],
        "Glass": [
            "Glass jar", "Glass bottle", "Glass cup", "Broken glass"
        ],
        "Waste": [
            "Cigarette", "Food waste", "Foam cup",
            "Disposable food container", "Foam food container",
            "Shoe", "Unlabeled litter", "Styrofoam piece"
        ],
        "Battery": [
            "Battery"
        ],
    }

In [2]:
model = ObjectDetectionModel(num_classes=len(grouped_classes), num_anchors=4, grid_size=4)
model.count_parameters()
coco_processor = COCOProcessor(classes=grouped_classes)

MODEL PARAMETER SUMMARY
Total parameters:      390,700
Trainable parameters:  390,700
Non-trainable params:  0


In [3]:
extracted_trash = coco_processor.extract_annotations(
    'D:/Sakal/AI_FARM/Recycling_Classification/Dataset/Dataset/Trash Detection.v14i.coco/train/_annotations.coco.json',
    'D:/Sakal/AI_FARM/Recycling_Classification/Dataset/Dataset/Trash Detection.v14i.coco/train',
    convert=False
)

extracted_taco = coco_processor.extract_annotations(
    'D:/Sakal/AI_FARM/Recycling_Classification/Dataset/TACO/data/annotations.json',
    'D:/Sakal/AI_FARM/Recycling_Classification/Dataset/TACO/data',
    convert=True
)

classes_names_trash = []
for label in extracted_trash:
    classes_names_trash.extend(label['Class'])
classes_names_trash = list(set(classes_names_trash))

classes_names_taco = []
for label in extracted_taco:
    classes_names_taco.extend(label['Class'])
classes_names_taco = list(set(classes_names_taco))

In [4]:
from PIL import Image
classes = [item for item, value in grouped_classes.items()] # ['Metal', 'Plastic', 'Paper', 'Glass', 'Waste', 'Battery']

processor = TrainingProcessor(
    input_size=448,
    grid_size=model.grid_size,
    num_anchors=model.num_anchors,
    classes=classes,
)

trash_dataset = TrainingDataset(data_json=extracted_trash, processor=processor, is_training=False)
trash_dataloader = DataLoader(trash_dataset, batch_size=40, shuffle=True)

# image_tensor, target_tensor, anchor_pose = processor.process_training_sample(
#     extracted_trash[90], apply_augmentation=False, get_anchors=True)

# # processor.visualize_training_sample(
# #     image_tensor, target_tensor, anchor_pose,)

# bboxes = processor.convert_yolo_output_to_bboxes(target_tensor, grid=True, class_tensor=True, is_training=True)
# processor.draw_bbox_on_image(image_tensor, bboxes, tensor=True, show=False)

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# criterion = ObjectDetectionLoss(processor=processor, classes_alpha=[0.3,0.4,0.3,0.1,0.3,0.2])
# criterion = ObjectDetectionLoss(processor=processor, classes_alpha=[0.3,0.4,0.3,0.1,0.3,0.2])
criterion = create_balanced_loss(processor)
optimizer = optim.Adam(
    model.parameters(),
    lr=1e-4,           # learning rate
    betas=(0.9, 0.999),# beta1 and beta2 for momentum estimates
    eps=1e-8,          # small constant for numerical stability
    weight_decay=0     # L2 regularization
)

num_epochs = 50
batch_interval = 75

In [6]:
monitor = train_with_monitoring(
    model=model,
    dataloader=trash_dataloader,
    loss_fn=criterion,
    optimizer=optimizer,
    num_epochs=50,
    # scheduler=scheduler,
    save_model_path='best_model.pth',
    monitor_frequency=200, 
    device='cuda'
)

Training log loaded from detection_training.json
Starting training with enhanced monitoring...

TRAINING STATUS - Epoch 0, Batch 0
📈 CURRENT METRICS:
   total_loss  : 11.3347 (overall: 13.7867) ➡️
   box_loss    : 5.0000 (overall: 5.0000) ➡️
   obj_loss    : 1.6320 (overall: 4.0532) ➡️
   cls_loss    : 0.5634 (overall: 0.7058) ➡️

🔍 LEARNING DYNAMICS:
   Variance:     1.166505 (changing)
   Trend:        -0.140719 (improving)
   Best loss:    11.0620 (epoch 0)

📊 CURRENT PHASE: STABLE

💡 RECOMMENDATIONS:
   ✅ Training progressing well!
🔄 Adaptive weights updated:
   Box: 2.021, Obj: 0.491, Cls: 2.391
🔄 Adaptive weights updated:
   Box: 2.040, Obj: 0.663, Cls: 3.197
🔄 Adaptive weights updated:
   Box: 2.059, Obj: 0.819, Cls: 3.876
📊 Training phase: increasing

🎯 Epoch 1/50 completed:
   Average Loss: 14.9334
   Best Loss: 11.0620 (epoch 1)

TRAINING STATUS - Epoch 1, Batch 0
📈 CURRENT METRICS:
   total_loss  : 16.0560 (overall: 14.3843) 📈
   box_loss    : 5.0000 (overall: 5.0000) ➡️
   

KeyboardInterrupt: 

In [ ]:
# model.to(device)
# training_lifetime_loss = 0.0
# training_lifetime_batch = 0

# for epoch in range(num_epochs):
#     model.train()  # Ensure model is in training mode
#     epoch_loss = 0.0
#     batch_interval_loss = 0.0
#     num_batches = 0
    
#     for i, (x, y) in enumerate(trash_dataloader):
#         x, y = x.to(device), y.to(device)
        
#         optimizer.zero_grad()
#         y_pred = model(x)
#         loss = criterion(y_pred, y)['total_loss']
        
#         # Check for invalid loss values
#         if torch.isnan(loss) or torch.isinf(loss):
#             print(f"Warning: Invalid loss detected at epoch {epoch+1}, batch {i}")
#             print(f"Loss value: {loss.item()}")
#             continue  # Skip this batch
        
#         loss.backward()
        
#         # Add gradient clipping for stability (especially important for your loss function)
#         torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=10.0)
        
#         optimizer.step()
        
#         # Accumulate losses
#         loss_item = loss.item()
#         epoch_loss += loss_item
#         batch_interval_loss += loss_item
#         training_lifetime_loss += loss_item
#         num_batches += 1
#         training_lifetime_batch += 1
        
#         # Print batch interval statistics
#         if i % batch_interval == 0 and i != 0:
#             avg_interval_loss = batch_interval_loss / batch_interval  # Fixed division
#             avg_epoch_loss_so_far = epoch_loss / num_batches
#             avg_lifetime_loss = training_lifetime_loss / training_lifetime_batch
            
#             print(f'\tBatch: [{i}/{len(trash_dataloader)}], '
#                   f'Interval Loss: {avg_interval_loss:.4f}, '
#                   f'Epoch Loss: {avg_epoch_loss_so_far:.4f}, '
#                   f'Lifetime Loss: {avg_lifetime_loss:.4f}')
            
#             batch_interval_loss = 0.0
    
#     # Epoch summary
#     if num_batches > 0:  # Avoid division by zero
#         avg_epoch_loss = epoch_loss / num_batches
#         avg_lifetime_loss = training_lifetime_loss / training_lifetime_batch
        
#         print(f'Epoch: [{epoch+1}/{num_epochs}], '
#               f'Avg Epoch Loss: {avg_epoch_loss:.4f}, '
#               f'Avg Lifetime Loss: {avg_lifetime_loss:.4f}')
        
#         # Optional: Save checkpoint periodically
#         if (epoch + 1) % 25 == 0:  # Save every 5 epochs
#             checkpoint = {
#                 'epoch': epoch + 1,
#                 'model_state_dict': model.state_dict(),
#                 'optimizer_state_dict': optimizer.state_dict(),
#                 'loss': avg_epoch_loss,
#                 'lifetime_loss': avg_lifetime_loss
#             }
#             torch.save(checkpoint, f'checkpoint_epoch_{epoch+1}.pth')
#             print(f"Checkpoint saved at epoch {epoch+1}")

# print("Training completed!")
# print(f"Final average lifetime loss: {training_lifetime_loss / training_lifetime_batch:.4f}")

Grid-based matching failed: 'FixedObjectDetectionLoss' object has no attribute '_compute_grid_based_loss', falling back to direct matching
	Batch: [75/150], Interval Loss: 1.5043, Epoch Loss: 1.4845, Lifetime Loss: 1.4845
Epoch: [1/50], Avg Epoch Loss: 1.5115, Avg Lifetime Loss: 1.5115
	Batch: [75/150], Interval Loss: 1.8156, Epoch Loss: 1.7917, Lifetime Loss: 1.6057
Epoch: [2/50], Avg Epoch Loss: 1.9637, Avg Lifetime Loss: 1.7376
	Batch: [75/150], Interval Loss: 2.2849, Epoch Loss: 2.2548, Lifetime Loss: 1.8421
Epoch: [3/50], Avg Epoch Loss: 2.2099, Avg Lifetime Loss: 1.8950
	Batch: [75/150], Interval Loss: 2.2155, Epoch Loss: 2.1863, Lifetime Loss: 1.9371
Epoch: [4/50], Avg Epoch Loss: 2.1500, Avg Lifetime Loss: 1.9588
	Batch: [75/150], Interval Loss: 2.0564, Epoch Loss: 2.0294, Lifetime Loss: 1.9667
Epoch: [5/50], Avg Epoch Loss: 2.0719, Avg Lifetime Loss: 1.9814
	Batch: [75/150], Interval Loss: 2.2168, Epoch Loss: 2.1876, Lifetime Loss: 2.0004
Epoch: [6/50], Avg Epoch Loss: 2.1570,

In [ ]:
loss['total_loss']

tensor([5.1836], device='cuda:0', grad_fn=<AddBackward0>)

In [ ]:
criterion.binary_focal_loss.alpha

0.25